Prepare for end of july 29 2016 submission. Is this Y4Q4?


In [1]:
import pandas
import gcat
import paramiko
from IPython import display

In [2]:
from curation_common import *

In [3]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [4]:
spreadsheet_name = 'ENCODE Submission for Y4Q1'

# Register Biosamples

In [5]:
sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosample = sheet.parse('Biosamples', header=0)
created = server.post_sheet('/biosamples/', biosample, verbose=True, dry_run=True)
print(len(created))
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

0


# Register Experiments

In [6]:
sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = sheet.parse('Experiments', header=0)

created = server.post_sheet('/experiments/', experiments, 
                            verbose=True, 
                            dry_run=True)
print(len(created))
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)
    display.display_pretty(created)

0


# Register Libraries

In [7]:
sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = sheet.parse('Libraries', header=0)

created = server.post_sheet('/libraries/', libraries, 
                            verbose=True, 
                            dry_run=True)
print(len(created))
if created:
    display.display_pretty(created)
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

0


# Register Replicates

In [8]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = book.parse('Replicates', header=0)

created = server.post_sheet('/replicates/', replicates, 
                            verbose=True, 
                            dry_run=True)
print(len(created))
if created:
    display.display_pretty(created)
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

0


# Create Manifest

In [11]:
def make_manifest(spreadsheet_name, submission_name, submission_root, map_name):
    book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
    libraries = book.parse('Libraries')
    replicates = book.parse('Replicates')
    
    pongo = paramiko.SSHClient()
    pongo.set_missing_host_key_policy(paramiko.WarningPolicy())
    pongo.connect('pongo.cacr.caltech.edu', username='diane', 
                  look_for_keys=False,
                  #gss_auth=True, gss_kex=True
                 )
    sftp = pongo.open_sftp()
    sftp.chdir(submission_root)
    files = 0
    for line in sftp.open(map_name):
        # skip comments
        line = line.strip()
        if len(line) == 0 or line.startswith('#'):
            continue

        library_id, dirname = line.split()
        alias = 'barbara-wold:{}'.format(library_id)
        library_row = libraries[libraries['aliases:array'] == alias]
        replicate_row = replicates[replicates['library'] == alias]
        for filename in sftp.listdir(dirname):
            if filename.endswith('fastq.gz'):
                relative_path = os.path.join(dirname, filename)
                metadata = {
                    'dataset': list(replicate_row['experiment'])[0],
                    'submitted_file_name': relative_path,
                    'replicate': list(replicate_row['uuid'])[0],
                    'lab': list(library_row['lab'])[0],
                    'award': list(library_row['award'])[0],
                }
                #display.display_pretty(metadata)
                print('<http://jumpgate.caltech.edu/wiki/SubmissionsLog/{}#{}>'.format(submission_name, dirname))
                print('  encode3:dataset "{}" ;'.format(metadata['dataset']))
                print('  encode3:replicate "{}" ;'.format(metadata['replicate']))
                print('  encode3:lab "{}" ;'.format(metadata['lab']))
                print('  encode3:award "{}" .'.format(metadata['award']))

                print('<file:///woldlab/castor/home/diane/proj/submission/{}/{}>'.format(submission_name, relative_path))
                print('  encode3:output_type "reads" ;')
                print('  encode3:file_format "fastq" .')
                files += 1

    print('generated {} records'.format(files))


In [12]:
make_manifest(spreadsheet_name, 'encode-20160729', '/home/diane/proj/submission/encode-20160729/', 'libs.tsv')

/usr/lib/python3/dist-packages/paramiko/client.py:658: UserWarning: Unknown ssh-rsa host key for pongo.cacr.caltech.edu: b'6833531597d6176b45c27f33e9406a12'
  (key.get_name(), hostname, hexlify(key.get_fingerprint())))


<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-20160729#17288-mouse-e10.5-forebrain-rep1>
  encode3:dataset "ENCSR304RDL" ;
  encode3:replicate "013e43ea-7897-4742-b4ef-7ad441f7bd02" ;
  encode3:lab "barbara-wold" ;
  encode3:award "U54HG006998" .
<file:///woldlab/castor/home/diane/proj/submission/encode-20160729/17288-mouse-e10.5-forebrain-rep1/17288_HT2GVBCXX_c116_l1.fastq.gz>
  encode3:output_type "reads" ;
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-20160729#17288-mouse-e10.5-forebrain-rep1>
  encode3:dataset "ENCSR304RDL" ;
  encode3:replicate "013e43ea-7897-4742-b4ef-7ad441f7bd02" ;
  encode3:lab "barbara-wold" ;
  encode3:award "U54HG006998" .
<file:///woldlab/castor/home/diane/proj/submission/encode-20160729/17288-mouse-e10.5-forebrain-rep1/17288_HT2GVBCXX_c116_l2.fastq.gz>
  encode3:output_type "reads" ;
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-20160729#17289-mouse-e10.5-forebra